In [4]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

import requests
import pymongo

from flask import Flask, render_template, redirect
#from flask_pymongo import PyMongo
app = Flask(__name__)

app.config["MONGO_URI"] = "mongodb://localhost:27017/mars_app"

@app.route("/")
def index():
    mars=mongo.db.find_one()
    return render_template("index.html", mars=mars)


In [5]:
# Path to chromedriver
#!which chromedriver

In [6]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\jbate\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [7]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [8]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [9]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8848/nasas-perseverance-pays-off-back-home/" target="_self">NASA's Perseverance Pays Off Back Home</a></div>

In [10]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Perseverance Pays Off Back Home"

In [11]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

'Even as the Perseverance rover approaches Mars, technology on board is paying off on Earth.'

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [18]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [19]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [20]:
# Define database and collection
db = client.mars_db
collection = db.images

In [21]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.


item = browser.find_by_css("a.product-item h3")

for i in range(len(item)):
    hemisphere = {}
    browser.find_by_css("a.product-item h3")[i].click()
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    hemisphere_image_urls.append(hemisphere)
    
    browser.back()
    

In [22]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls



[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [23]:
# 5. Quit the browser
browser.quit()